# High-Frequency Effects and Modelling on FX Market in Response to US Macroeconomic Data Releases

## Steps:
1. Research and Planning
    * Define clear research goals and objectives
    * Decide on macroeconomic events and FX pairs
2. Literature Review
    * Review past papers
    * Identify gaps
3. Data Collection and Processing
    * Identify data sources
    * Collect FX pairs (date, minute interval, 
    * Collect macroeconomic events (date, expected value, actual value)
    * Merge datasets
    * Do some analysis for data dimensionality (e.g., analysis whether to keep 10 minutes before and 10 minutes after only of the data release)
    * Keep only data with the macro events (not whole month but only on data releases)
4. Exploratory Analysis
    * Correlation of price movement with each macro event expected vs actual
    * Feature importance (which macro event brings more volume)
5. Model Development
    * Create some either ABM or probabilistic TS model based on historical data
6. Backtesting and Market Simulation
    * KPIs and statistical measurement of model on test data (can be same as past)
    * Create a class/function to take input of a pair, expected value of macro data, and minute after event to give some price range and classification with confidence interval
* Results and Discusion
    * Interpret market reactions to economic data
    * Compare model accuracy with benchmarks

----------------------

## Libraries

In [1]:
# Libraries for data fetching
import yfinance as yf
from polygon import RESTClient
from fredapi import Fred

# Libraries for data analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

----------------------

## Data Retrieval
* Data Collection FX pair tickers (symbols) for **Yahoo Finance**: 
    * **DXY** 🠆 "DX-Y.NYB",
    * **USD/GBP** 🠆 "GBPUSD=X",
    * **USD/EUR** 🠆 "EURUSD=X",
    * **USD/CNY** 🠆 "CNY=X"


* Data Collection FX pair tickers (symbols) for **Polygon.io**:
    * **USD/EUR** 🠆 C:USD/EUR
    * **USD/GBP** 🠆 C:USD/GBP
    * **USD/CNY** 🠆 C:USD/CHF
    
* Data Collection FX pair tickers (symbols) for **fred.stlouisfed.org**:
    * **Inflation** 🠆 CPIAUCSL
    * **Interest Rates** 🠆 FEDFUNDS
    * **GDP** 🠆 GDPC1
    * **Unemployment Rate** 🠆 UNRATE
    * **PMI** 🠆 NAPM
    * **DXY** 🠆 DTWEXBGS

In [2]:
def yf_get_fx_data(fx_pair, interval, start_date, end_date):
    """
    Retrieve historical data for a given FX pair from Yahoo Finance. 

    Parameters:
    - fx_pair (str): The FX pair to retrieve data for FX pairs and DXY (USD Index).
    - interval (str): The interval for the data (e.g., '1m', '1d', '1wk', '1mo').
    - start_date (str): The start date in 'YYYY-MM-DD' format.
    - end_date (str): The end date in 'YYYY-MM-DD' format.

    Returns:
    - pd.DataFrame: A DataFrame containing the historical data.
    """
    try:
        # Fetch data using yfinance in the form of 'Year-Month-Date Minutes'
        data = yf.download(fx_pair, start=start_date, end=end_date, interval=interval)
        
        # Modify datetime and set it as index
        data.index = data.index.strftime('%Y-%m-%d %H:%M')
        
        # Column manipulation
        data.columns = data.columns.droplevel(1) # Drop the Ticker Column level
        data = data.rename_axis(f'{fx_pair}', axis=1) # Rename axis 1
        data.index = pd.to_datetime(data.index)
        data = data[["Open", "High", "Low", "Adj Close"]]

        return data
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


def polygon_get_fx_data(fx_pair, interval, start_date, end_date, api_key='0GbCWQKVtPfEMJvPmc1n50psE3zsW1c8'):
    """
    Retrieve historical FX data from Polygon.io.

    Parameters:
    - interval (str): The interval for the data ('1m', '5m', '1h', '1d').
    - start_date (str): The start date in 'YYYY-MM-DD' format.
    - end_date (str): The end date in 'YYYY-MM-DD' format.
    - api_key (str): Polygon.io API key.

    Returns:
    - pd.DataFrame: A DataFrame containing historical USD/CNY forex data.
    """
    try:
        # Bring the fx pair ticker to Polygon's format
        fx_pair_polygon_formated = "C:" + str(fx_pair)

        client = RESTClient(api_key)

        # Request data
        data_request = client.get_aggs(
            ticker=fx_pair_polygon_formated, 
            multiplier=1, 
            timespan=interval,
            from_=start_date, 
            to=end_date
        )
    
        # Convert response to DataFrame
        df = pd.DataFrame(data_request)
        
        # Check if data is returned
        if df.empty:
            print("No data found.")
            return None

        # Convert timestamp to datetime format and set Datetime as an index
        df["Datetime"] = pd.to_datetime(df["timestamp"], unit="ms")
        df.set_index("Datetime", inplace=True)
        
        # Drop columns not needed
        df = df.drop(columns=["otc", "transactions", "timestamp"])
        
        # Rename columns to align with yf for the feature engineering and merging later on
        df = df.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Adj Close", "volume": "Volume"})
        df = df.rename_axis(f'{fx_pair}', axis=1)
        
        return df

    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    
    
def fetch_fred_data(indicator, start_date, end_date, api_key = "134a864b086b836aabbdeb1fc9022c5a"):
    """
    Fetches historical economic data from FRED based on the selected economic indicator for given period.

    Parameters:
    - series_id (str): The FRED series ID (e.g., "UNRATE" for Unemployment Rate).
    - start_date (str): The start date in 'YYYY-MM-DD' format.
    - end_date (str): The end date in 'YYYY-MM-DD' format.
    - api_key (str): Your FRED API key.

    Returns:
    - pd.DataFrame: A DataFrame containing the requested economic data with percentage change. 
                    If CPI data is requested then Year Over Year difference is presented as well.
    """
    try:
        # Initialize FRED API
        fred = Fred(api_key=api_key)

        # Fetch data
        data = fred.get_series(indicator)

        # Convert to DataFrame
        df = pd.DataFrame(data, columns=[indicator])
        df.index.name = "Date"

        # Filter based on start and end date
        df = df.loc[start_date:end_date]
        
        # Calculate percentage change from previous row
        df["Percentage_Change"] = (df[indicator].pct_change()).round(4)
        
        # 12-month YoY inflation
        if indicator == "CPIAUCSL":
            df["YoY_Inflation"] = (df[indicator].pct_change(12) * 100).round(2)

        return df

    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

----------------------

## Feature Engineering

In [3]:
def calculate_volatility(df):
    """
    Calculate the average volatility dynamically based on the number of rows in the DataFrame.

    Parameters:
    - df (pd.DataFrame): FX data.

    Returns:
    - pd.DataFrame: The original DataFrame with added 'Volatility', 'Average_Volatility', and 'Volatility_Multiplier' columns.

    """

    if df is None or df.empty:
        print("Error: DataFrame is empty or invalid.")
        return None

    # Calculate Log Returns
    df['Log_Returns'] = np.log(df['Adj Close'] / df['Adj Close'].shift(1))
    # Compute Volatility
    df['Volatility'] = df['Log_Returns'].abs()
    # Compute Average Volatility based on number of rows
    df['Average_Volatility'] = df['Volatility'].mean()
    # Compute Volatility Multiplier
    df['Volatility_Multiplier'] = round(df['Volatility'] / df['Average_Volatility'],2)

    return df

------------

## Plotting Functions

In [4]:
def plot_time_series(df, column):
    """
    Plots a time series for a given data and column.

    Parameters:
    - df (pd.DataFrame): DataFrame containing time series data with a Datetime index.
    - column (str): Column name to plot.
    - title (str): Title of the plot (default: "Time Series Plot").

    Returns:
    - None (Displays the plot)
    """
    
    if column not in df.columns:
        print(f"Error: Column '{column}' not found in DataFrame.")
        return None
    
    fx_pair = df.columns.name if df.columns.name else "FX Pair"
    # Create plot
    fig, ax = plt.subplots(figsize=(12, 6))
    df[column].plot(ax=ax)
    
    # Improve x-axis readability
    ax.set_xlabel("Datetime", fontsize=12)
    ax.set_ylabel(column, fontsize=12)
    ax.set_title(f"'{fx_pair}' {column} Over Time", fontsize=14)
    ax.tick_params(axis='x', rotation=0)
    
    # Ensure the x-axis starts and ends exactly at the first and last data points (applicable only to yf data)
    if "vwap" not in df.columns:
        # Format x-axis to time
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
        ax.xaxis.set_major_locator(mdates.AutoDateLocator())
        ax.set_xlim(df.index.min(), df.index.max())

    ax.grid(True, which="major", linestyle="-", linewidth=0.6, alpha=1)  # Major grid
    ax.grid(True, which="minor", linestyle="-", linewidth=0.5, alpha=1)  # Minor grid

    plt.show()
    pass

----------------------

## Testing functionalities

In [9]:
# YF data
# fx_data = calculate_volatility(yf_get_fx_data('USDGBP=X', '1m', '2025-01-15', '2025-01-16'))

# Polygon data
# polygon_fx_data = calculate_volatility(polygon_get_fx_data("USDGBP", "minute", "2025-01-15", "2025-01-15"))

In [6]:
# fx_data
# polygon_fx_data

In [7]:
# plot_time_series(fx_data, column="Adj Close") # YF
#plot_time_series(polygon_fx_data, "Adj Close") # Polygon

-----------

## 1. Daily Volume per Year

Bloomberg terminal needed for data depth per year and also volume

## 2. Volume per Country
Bloomberg terminal needed for data depth per year and also volume

## 3. Economic Data Seasonality

* **CPI** monthly (10-15 on each month)
* **IR** 6 weeks
* **GDP** Quarterly
* **UNRATE** monthly (every 1st Friday)
* **PMI** monthly (first business day)

Bloomberg terminal needed for:
1. Exact Dates for each event listed above for the past 10 years
2. Expected Value vs Actual Value (surprise)

In [8]:
# Example Usage
economic_data = fetch_fred_data(indicator="CPIAUCSL", start_date="2015-01-01", end_date="2025-01-01")
# economic_data

## 4.  Economic Announcment Selection - Feature Importance / Importance of Macro events (feature importance on movement based on price change and volatility)

1. Do some feature importance to keep the major macro data releases and do detailed analysis on these ones.

## 5. Responses to CPI, IR, GDP, Unemployment Rate (R2 before appendix) - Jump (R4 p32) - Volatility Increase on each event table - Data Release Trend: Before/On/After

1. Charts
2. Table with all info for CPI, IR, GDP, UR, JUMP, Vol increase from avg for all pairs

## 6. Prior vs Now (volatility and importance)

1. Bar chart showing importance % in the past for the price movement for all events (x axis) on DXY:
    * For now pick average of 1 or 2 last years of data (e.g., 2023-2024)
    * For past pick average of 1 or 2 years in the past data (e.g., 2018-2019)

## 7. Analysis of High Volume Intraday on Data Releases
1. Find out in a way how much to keep out of each day (e.g., 10 minutes before and 10 minutes after the event)
2. Volatility within -5m/-1m, 1m, 1m-5m, >5m (% to understand when the market moves the most)

## 8. Correlation of USD pairs and DXY price movements

1. Correlation heatmap
2. Identify which one moves first if not all together to see for arbritrage opportunity

-------

# <span style="color:blue">Bloomberg Terminal Data</span>
1. **Last 7 years (2018-2025) of data with datetime and Expected vs Actual for:** (calculate surprise as well)
    * IR
    * CPI
    * GDP
    * PMI
    * UNRATE  
<br>
<br>

2. **Minutes for all the single macro events above with volume if possible**
    * USD/GBP
    * USD/EUR
    * USD/CNY
    * DXY
    
    
## Things to check
1. Can I get FX minute data for 2018 for example?